### Importação de Bibliotecas

In [ ]:
from IPython.display import clear_output
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer
import numpy as np
import pandas as pd
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import matplotlib.pyplot as plt
import speech_recognition as sr
import pyttsx3
import os
import sys
# Redireciona a saída padrão de erro para /dev/null
sys.stderr = open(os.devnull, 'w')

### Importação da tabela base de dados e base de perguntas & respostas

In [ ]:
df = pd.read_excel("QA_chatbot/database.xlsx")
df["Data"] = pd.to_datetime(df["Data"])

qna = pd.read_excel("QA_chatbot/QNA.xlsx")

bot = ChatBot('B-Bot')
bot.storage_adapter='chatterbot.storage.SQLStorageAdapter'

### Funções de pré-processamento de texto

In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('portuguese')]
    return ' '.join(a)

# pre-process
def finalpreprocess(string):
    return stopword(preprocess(string))


### Função para trainar o bot

In [ ]:
def train_bot():
    df = pd.read_excel("QA_chatbot/database.xlsx")
    df["Data"] = pd.to_datetime(df["Data"])

    qna = pd.read_excel("QA_chatbot/QNA.xlsx")

    # questions cleaning database
    qna['clean_pergunta'] = qna['pergunta'].apply(lambda x: finalpreprocess(x))
    qna['clean_resposta'] = qna['resposta'].apply(lambda x: finalpreprocess(x))

    list_train = []
    corpus_trainer = ChatterBotCorpusTrainer(bot)
    corpus_trainer.train("chatterbot.corpus.portuguese")

    for index, row in qna.iterrows():
        list_train.append(row.clean_pergunta)
        list_train.append(row.resposta)

    trainer = ListTrainer(bot)

    trainer.train(list_train)

### Funções de conversão Texto / 

In [ ]:
def ConvertTexto(texto):
    engine = pyttsx3.init("espeak")
    engine.setProperty("rate",100)
    engine.setProperty('volume',0.2)
    engine.setProperty("voice","mb-br4")
    engine.say(texto)
    engine.runAndWait()
    
def ConvertVoz():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        
        r.adjust_for_ambient_noise(source)
        print("PODE FALAR\n")
        audio = r.listen(source)
        print("Você disse...")
      
        try:
            text = r.recognize_google(audio,language='pt-BR')
            #print(text)
        except sr.UnknownValueError:
            #print("Poxa, eu não entendi")
            text = "Poxa, eu não entendi!"
        except sr.RequestError as e:
            print("Error {0}".format(e))
            text = "Eita houve algum erro"
            
    return(str(text))

### Função para adicionar um comando em python

In [ ]:
def AprendePython():
    
    qna = pd.read_excel("QA_chatbot/QNA.xlsx")
    t = "Legal, vamos lá!"
    ConvertTexto(t)
    name = input("\nDigite a sua pergunta (como descreveria a função):  ")
    pyc = input("\nDigite o comando em python corresponde à sua pergunta:  ")
    
    try:
        print("\n")
        print(eval(pyc))
        input("O resultado do comando digitado foi apresentado.\nPressione qualquer tecla para prosseguir...")
        print("\n")
        
        new_row = pd.DataFrame([[str(name),str(pyc)]],columns=['pergunta','resposta'])
        qna = qna.append(new_row,ignore_index=True)
        print(qna.tail())
        qna.to_excel('QA_chatbot/QNA.xlsx')  
        train_bot()
        
        print("\nOk, agora eu aprendi!\n")
        t = "Ok, agora eu aprendi!"
        ConvertTexto(t)
        input("Digite qualquer coisa para voltar ao menu...")
    
    except Exception as e:
        print("\nHouve um erro:")
        print(e)
        t = "Xi... algo deu errado no aprentizado!"
        ConvertTexto(t)
        input("Digite qualquer coisa para retornar...")       

### Função de analisar a pergunta

In [ ]:
def AnalisaPergunta(pergunta):
    
    result = finalpreprocess(pergunta)
    resposta = bot.get_response(result)
    
    if float(resposta.confidence) > 0.5:
        print("Comando: "+str(resposta))
        t = "Aqui está seu resultado!"
        ConvertTexto(t)
        try:
            ans = eval(str(resposta))
            print(ans)
            
            ans["Data"] = ans["Data"].astype("datetime64")
            ans = ans.set_index("Data")
            
            plt.plot(ans, marker='o')
            plt.show()
            
            input("Digite qualquer coisa para continuar.......")          
            
            print("A resposta está correta? Por favor, responda 'Sim' ou 'Não'")
            t = "A resposta está correta?"
            ConvertTexto(t)
            ans = ConvertVoz()
            
            if ans == 'sim' or ans == 'Sim' or ans == 'esta' or ans == 'está':
                print("\n Muito obrigada pelo feedback!!!")
            
            elif ans == 'não' or ans == 'Não':
                print("\nPreciso aprender a resolver essa pergunta.")
                t = "Desculpe, mas eu ainda não sei responder esta pergunta. Me ajude!"
                AprendePython()
                
        except:
            print(ans)

                    
    else:
        print("\nPreciso aprender a resolver essa pergunta. Pode me ajudar? Por favor, responda 'Sim' ou 'Não'")
        t = "Desculpe, mas eu ainda não sei responder esta pergunta. Pode me ajudar?"
        ConvertTexto(t)
        
        ans = ConvertVoz()        
        if (ans == "sim"):
            AprendePython()
        else:
            print("Tudo bem, obrigada!")
            t = "Tudo bem, obrigada!"
            ConvertTexto(t)
            input("Digite qualquer coisa para voltar ao menu...")


### Função da opção 1

In [ ]:
def solicitacao():
    print("IMPORTANTE: para OEE diga Eficiência 'Ex: Eficiência da máquina 1 em janeiro'")
    print("ATENÇÃO: caso queira uma dica, diga o seguinte: 'o que eu posso perguntar?'\n")
    t = "Considerando as dicas na tela, o que você deseja saber?"
    ConvertTexto(t)
    print(list(df.columns))
    
    print("\nO que deseja saber?")
            
    pergunta = ConvertVoz()
    print(pergunta)
        
    if pergunta == "o que eu posso perguntar":            
        t = "Essas são as questões que eu já sei responder:"
        print("\nQuestões que já sei responder:\n")
        ConvertTexto(t)            
        print(qna['pergunta'])
            
    else:
        print("\nANALISANDO...")
        t = "Aguarde que vou analisar sua pergunta!"
        ConvertTexto(t)
        AnalisaPergunta(pergunta)

### Função da Opção 5

In [ ]:
def conversa(nome):
    print('Olá, o que deseja saber?')
    t = "Olá "+str(nome)
    ConvertTexto(t)
    t = "O que deseja saber?"
    ConvertTexto(t)
    
    pergunta = ConvertVoz()
    result = finalpreprocess(pergunta)
    resposta = bot.get_response(result)
        
    if float(resposta.confidence) > 0.5:
        t = str(resposta)
        ConvertTexto(t)
           
    else:
        print("\nNão sei, você pode me ajudar a descobrir? Responda 'Sim' ou 'Não'")
        t = "Pode me ajudar a descobrir?"
        ConvertTexto(t)
        
        ans = ConvertVoz()        
        if (ans == "sim"):
            AprendePython()
        else:
            print("Tudo bem, obrigada!")
            t = "Tudo bem, obrigada!"
            ConvertTexto(t)
            input("Digite qualquer coisa para voltar ao menu...")
  

### Loop do chatbot

In [ ]:
train_bot()
print("\n **************************  B-CHAT  ************************************* \n")

print("Oi, eu sou o B-Chat \n")
t = "Oi, eu sou o Bi-Chét"
ConvertTexto(t)

print("Por favor, fale somente quando aparecer a palavra FALE no seu console, ok? \n")
t = "Por favor, fale somente quando aparecer a palavra FALE no seu console, ok?"
ConvertTexto(t)

print("Me diga: Qual é o seu nome? \n")
t = "Me diga: Qual é o seu nome?"
ConvertTexto(t)
nome = ConvertVoz()
print("\nOlá, "+str(nome)+" esperamos que esteja bem!\n")
t = "Olá, "+str(nome)+" esperamos que esteja bem!"
ConvertTexto(t)

a=1
while a==1:

    print("Na base de dados disponível temos os seguintes dados (colunas no dataframe):")    
    t = "Na base de dados disponível temos os seguintes dados:"
    ConvertTexto(t)
    print(list(df.columns))    
    print("\nComo posso te ajudar?")
    t = "Vou mostrar o menu e você pode digitar uma opção para eu ajudar?"
    ConvertTexto(t)
    
    menu = input('Digite: \n'+
                    ' 1 - para solicitar um gráfico a partir do DataFrame \n'+
                    ' 2 - para enviar um comando python para ser salvo \n'+
                    ' 3 - para me treinar novamente \n'+
                    ' 4 - para uma conversa aleatória \n'+
                    ' 5 - para encerrar o chat \n')
    clear_output(wait=True)
    
    if menu == '1':
        solicitacao()            
    elif menu == '2':
        AprendePython()        
    elif menu == '3':        
        train_bot()
    elif menu == '4':
        conversa(nome)    
    elif menu == '5':
        t = "Até mais!"
        ConvertTexto(t)
        a = 2            
    else:
        print("\nOpção inválida.")
        t = "Digite uma opção válida, por gentileza!"
        ConvertTexto(t)
        
    print("***************************************************************")